In [ ]:
#SELENIUM MODULES
from selenium import webdriver
from selenium.webdriver.common.by import By

#SECONDARY LIBRARIES
import pandas as pd
import numpy as np
import os
import re
import time
import ssl
import sys
import math
import traceback

#ALERTS DEACTIVATION
import warnings
warnings.filterwarnings('ignore')

#OWNNED PRIMARY MODULES
from funciones_primarias import validacion
from funciones_secundarias import barra_carga,scroll_down


'''Protocoloes SSL'''
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

'''
OBJET DRIVER 
    - Options
    - URL
'''

#Firefox Options
firefox_profile = webdriver.FirefoxOptions()
firefox_profile.set_preference("browser.download.folderList", 2)
firefox_profile.set_preference("browser.download.dir", 'descargas')
firefox_profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")

# Firefox navigator 
driver = webdriver.Firefox(options=firefox_profile)


#Driver Instance -  URL Access
PAGE_URL = 'https://www.youtube.com/watch?v=pgg7ciV1Vfk&t=2277s'

driver.get(PAGE_URL)


In [ ]:
''' 
VALIDATION

URL Acceso, navegation and capture of X.Path
Scroll Test and Waitin Strategie

'''

#Find the first box comment as point of reference in the web page

seccion_comments = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-comments/ytd-item-section-renderer/div[3]/ytd-comment-thread-renderer['

barra_carga(15)
driver.execute_script('scrollBy(0,550)') #firt scroll down

barra_carga(15)
validacion(driver,seccion_comments) #validation function

In [ ]:
''' 
MAIN FUNCTION

'''


#SCRAPING VARIABLES
comentarios_recorridos = list()
centinela = True
comment_externo = 1
comment_interno = 1
base_final = pd.DataFrame()


while centinela == True:
    try:
        
        try:
            driver.find_element(By.XPATH, value= '/html/body/div[7]/div/div[1]/div[2]/div')
            print('Hay captcha...')
            time.sleep(30)
        except:
            print('No Captcha')
            
        # Level 1 comment principal
        level = 1
        # User NAME ubication
        try:
            display_name = driver.find_element(By.XPATH, value = seccion_comments + str(comment_externo) + "]/ytd-comment-renderer/div[3]/div[2]/div[1]/div[2]/h3").text
            print(display_name)
        except:
            print('Display NOT Found')
            break

        try:    
        # Username link
            username_yt = driver.find_element(by=By.PARTIAL_LINK_TEXT,value=display_name).get_attribute("href")
        # Principal comment
            comment = driver.find_element(By.XPATH, value = seccion_comments + str(comment_externo) + "]/ytd-comment-renderer/div[3]/div[2]/div[2]/ytd-expander/div/yt-formatted-string").text
        # Date
            date = driver.find_element(By.XPATH, value = seccion_comments + str(comment_externo) + "]/ytd-comment-renderer/div[3]/div[2]/div[1]/div[2]/yt-formatted-string/a").text
        
            print(str(comment) , str(username_yt), date)
            

        except:
            pass
   

        # Dataframe creation
        d = {"level": level, "user_name": display_name, "link_user":username_yt, "comment":comment,'date':date}
        base_temporal = pd.DataFrame(data=d,index=[comment_externo])
        base_final = pd.concat([base_final, base_temporal], ignore_index=True)
    
        comment_externo = comment_externo + 1 # adding one more value to the loop 
        
        driver.execute_script('scrollBy(0,250)')
        
        barra_carga(8)

    except:

        centinela = False
        print('FINISH')


base_final.to_excel('base_youtube_test_cuchicheo.xlsx') #exporting the dataframe to a excel file